In [1]:
from google.colab import userdata

API_KEY = userdata.get('OPENWEATHER_API_KEY')
print("✅ API key loaded successfully.")

✅ API key loaded successfully.


In [2]:
import requests
import pandas as pd
from datetime import datetime, timezone

LAT, LON = 24.8607, 67.0011  # Karachi

def get_realtime_data():
    try:
        # Weather API
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units=metric"
        weather = requests.get(weather_url, timeout=10).json()

        # Air Pollution API
        pollution_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={LAT}&lon={LON}&appid={API_KEY}"
        pollution = requests.get(pollution_url, timeout=10).json()

        # Validate structure
        if "main" not in weather or "list" not in pollution:
            raise ValueError("Invalid API response format")

        # Extract weather
        temp = weather["main"]["temp"]
        humidity = weather["main"]["humidity"]
        wind_speed = weather["wind"]["speed"]

        # Extract air quality
        air = pollution["list"][0]
        aqi = air["main"]["aqi"]
        comp = air["components"]

        data = {
            "datetime": datetime.now(timezone.utc),
            "city": "Karachi",
            "temp": temp,
            "humidity": humidity,
            "wind_speed": wind_speed,
            "aqi": aqi,
            "pm25": comp.get("pm2_5"),
            "pm10": comp.get("pm10"),
            "co": comp.get("co"),
            "no2": comp.get("no2"),
            "so2": comp.get("so2"),
            "o3": comp.get("o3"),
            # "nh3": comp.get("nh3")
        }

        return data

    except Exception as e:
        print(f"⚠️ Error fetching data: {e}")
        return None

In [3]:
data = get_realtime_data()
if data:
    df = pd.DataFrame([data])
    filename = "realtime_aqi_weather.csv"
    df.to_csv(filename, index=False)
    print(f"✅ Real-time data saved to {filename}")
    display(df)
else:
    print("❌ No data fetched.")

✅ Real-time data saved to realtime_aqi_weather.csv


,datetime,city,temp,humidity,wind_speed,aqi,pm25,pm10,co,no2,so2,o3
0,2025-10-13 18:05:17.706613+00:00,Karachi,27.07,50,3.09,3,29.52,75.05,131.14,0.14,1.17,102.01
